In [1]:
import pandas as pd
import os
import numpy as np
#!pip install tabula-py
#import tabula

# # Set the new directory path
# new_directory_path = 'C:/Users/Hernando/data/DDS_data'
# # Change the current working directory to the new directory
# os.chdir(new_directory_path)

# # Verify the change
# print(os.getcwd())

In [2]:
# List of Excel file paths
file_paths = ['Alta California.xlsx', 'San Andreas.xlsx', 'San Diego.xlsx','Valley Mountain.xlsx']

# Empty list to hold dataframes
dfs = []

# Loop through each file
for file_path in file_paths:
   # print(file_path)
    # Get list of sheet names
    xls = pd.ExcelFile(file_path)
    sheet_names = xls.sheet_names

    # Loop through each sheet in the workbook
    for sheet_name in sheet_names:
        #print(sheet_name)
        df = pd.read_excel(file_path, sheet_name=sheet_name)
        
        # Add columns for file and sheet identification
        df['Regional Center'] = file_path  # Or any other identifier
        df['Sheet Name'] = sheet_name

        # Append the dataframe to the list
        dfs.append(df)
#print(dfs)
# Concatenate all dataframes into one
combined_df = pd.concat(dfs, ignore_index=True)


In [3]:
def assign_category_titles(df):
    # Create a new column for the category title
    df['Title'] = None

    # Iterate over the DataFrame to identify the rows with 'Fiscal Year 2022-2023'
    for index, row in df.iterrows():
        if row['Unnamed: 1'] == 'Fiscal Year 2022-2023':
            # Capture the category title from the two rows above
            category_title = df.at[index-2, 'Unnamed: 1'] + ' ' + df.at[index-1, 'Unnamed: 1']
            
            # Assign this title to all rows on the same sheet
            sheet_name = row['Sheet Name']
            df.loc[df['Sheet Name'] == sheet_name, 'Title'] = category_title

    return df


def assign_age_categories(df):
    # Define the second level category titles
    age_categories = [
        "All Ages", 
        "For birth to age 2 years, inclusive", 
        "For age 3 years to 21 years, inclusive", 
        "For age 22 years and older"
    ]

    # Create a new column for the second level category titles
    df['Age Category'] = np.nan

    # Temporary variable to hold the current age category
    current_age_category = np.nan

    # Iterate over the DataFrame rows
    for index, row in df.iterrows():
        if row['Unnamed: 1'] in age_categories:
            # Update the current age category
            current_age_category = row['Unnamed: 1']
        
        # Assign the current age category to the new column
        df.at[index, 'Age Category'] = current_age_category
    
    return df


In [4]:
# Apply the function to the dataframe
combined_df = assign_category_titles(combined_df)
combined_df = assign_age_categories(combined_df)


In [5]:
columns_to_check = ['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7']
cleaned_df = combined_df.dropna(subset=columns_to_check, how='all')

# Step 1: Rename the columns
renamed_columns = {
    'Unnamed: 2': 'Consumers Count',
    'Unnamed: 3': 'Total Expenditures',
    'Unnamed: 4': 'Total Authorized Services',
    'Unnamed: 5': 'Per Capita Expenditures',
    'Unnamed: 6': 'Per Capita Authorized Services',
    'Unnamed: 7': 'Utilized'
}
cleaned_df.rename(columns=renamed_columns, inplace=True)

# Step 2: Transfer values for specific category title; Create new columns with NaN values
for col in ['Total Eligible Consumers', 'Consumers Receiving Purchased Services',
            'Consumers with No Purchased Services', 'Percent With No Purchased Services']:
    cleaned_df[col] = np.nan

# Define a mask for rows where 'Category Title' has the specified text
mask = cleaned_df['Title'] == 'Consumers with No Purchase of Services by Diagnosis - Legislative'

# Transfer the values and replace old ones with NaN
cleaned_df.loc[mask, 'Total Eligible Consumers'] = cleaned_df.loc[mask, 'Consumers Count']
cleaned_df.loc[mask, 'Consumers Receiving Purchased Services'] = cleaned_df.loc[mask, 'Total Expenditures']
cleaned_df.loc[mask, 'Consumers with No Purchased Services'] = cleaned_df.loc[mask, 'Total Authorized Services']
cleaned_df.loc[mask, 'Percent With No Purchased Services'] = cleaned_df.loc[mask, 'Per Capita Expenditures']

# Replace old column values with NaN for these specific rows
cleaned_df.loc[mask, ['Consumers Count', 'Total Expenditures', 
                       'Total Authorized Services', 'Per Capita Expenditures']] = np.nan

# Drop 'Unnamed: 0' column
cleaned_df.drop('Unnamed: 0', axis=1, inplace=True)

# Rename 'Unnamed: 1' to 'Category'
cleaned_df.rename(columns={'Unnamed: 1': 'Category'}, inplace=True)

# Drop rows with NaN in 'Category'
cleaned_df.dropna(subset=['Category'], inplace=True)

#remove '.xlsx' from Regional Center name
cleaned_df['Regional Center'] = cleaned_df['Regional Center'].str[:-5]

# Replace all '*' values with NaN 
for column in cleaned_df.columns:
    cleaned_df[column] = cleaned_df[column].replace('*', np.nan)
    

cleaned_df

C:\Users\Hernando\AppData\Local\Temp\ipykernel_2160\1065048956.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_df.rename(columns=renamed_columns, inplace=True)
C:\Users\Hernando\AppData\Local\Temp\ipykernel_2160\1065048956.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_df[col] = np.nan
C:\Users\Hernando\AppData\Local\Temp\ipykernel_2160\1065048956.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation

,Category,Consumers Count,Total Expenditures,Total Authorized Services,Per Capita Expenditures,Per Capita Authorized Services,Utilized,Regional Center,Sheet Name,Title,Age Category,Total Eligible Consumers,Consumers Receiving Purchased Services,Consumers with No Purchased Services,Percent With No Purchased Services
10,American Indian or Alaska\nNative,163,"$3,361,984","$4,489,724",20625.670123,27544.320552,0.748818,Alta California,POSxEth,Total Annual Expenditures and Authorized Servi...,All Ages,NaN,NaN,NaN,NaN
11,Asian,"3,369","$35,808,544","$55,766,125",10628.834817,16552.723268,0.64212,Alta California,POSxEth,Total Annual Expenditures and Authorized Servi...,All Ages,NaN,NaN,NaN,NaN
12,Black/African American,"3,678","$67,015,072","$94,283,572",18220.519859,25634.467675,0.710782,Alta California,POSxEth,Total Annual Expenditures and Authorized Servi...,All Ages,NaN,NaN,NaN,NaN
13,Hispanic,"5,832","$56,732,894","$88,281,428",9727.862483,15137.4191,0.642637,Alta California,POSxEth,Total Annual Expenditures and Authorized Servi...,All Ages,NaN,NaN,NaN,NaN
14,Native Hawaiian or Other\nPacific Islander,151,"$1,181,869","$1,904,961",7826.949007,12615.637616,0.620416,Alta California,POSxEth,Total Annual Expenditures and Authorized Servi...,All Ages,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8060,All other languages,NaN,NaN,NaN,5483.904,6840.62,0.801668,Valley Mountain,LANGxOther,Total Annual Expenditures and Authorized Servi...,For age 22 years and older,NaN,NaN,NaN,NaN
8061,Cantonese/Mandarin\nChinese,0,0,0,0,0,0,Valley Mountain,LANGxOther,Total Annual Expenditures and Authorized Servi...,For age 22 years and older,NaN,NaN,NaN,NaN
8062,English,56,"$1,012,980","$1,661,305",18088.922143,29666.164464,0.609749,Valley Mountain,LANGxOther,Total Annual Expenditures and Authorized Servi...,For age 22 years and older,NaN,NaN,NaN,NaN
8063,Spanish,NaN,NaN,NaN,836.415,1813.2,0.461292,Valley Mountain,LANGxOther,Total Annual Expenditures and Authorized Servi...,For age 22 years and older,NaN,NaN,NaN,NaN


In [6]:


# Function to reformat the title
def reformat_title(title):
    if 'by ' in title:
        parts = title.split('by ')
        new_title = f"{parts[1]}: {parts[0]}"
        return new_title
    else:
        return title

# Apply the function to the 'Title' column
cleaned_df['Title'] = cleaned_df['Title'].apply(reformat_title)

unique_titles = cleaned_df['Title'].unique()
print(unique_titles)

['Ethnicity or Race: Total Annual Expenditures and Authorized Services '
 'Language: Total Annual Expenditures and Authorized Services '
 'Diagnosis-Legislative: Total Annual Expenditures and Authorized Services '
 'Diagnosis-Summary: Total Annual Expenditures and Authorized Services '
 'Residence: Total Annual Expenditures and Authorized Services '
 'Diagnosis - Expanded: Total Annual Expenditures and Authorized Services '
 'Ethnicity or Race: Consumers with No Purchase of Services '
 'Language: Consumers with No Purchase of Services '
 'Residence: Consumers with No Purchase of Services '
 'Diagnosis - Legislative: Consumers with No Purchase of Services '
 'Diagnosis - Summary: Consumers with No Purchase of Services '
 'Ethnicity or Race: Total Annual Insurance-Related Expenditures '
 'Language: Total Annual Insurance-Related Expenditures '
 'Residence: Total Annual Insurance-Related Expenditures '
 'Diagnosis-Summary: Total Annual Insurance-Related Expenditures '
 'Diagnosis - Expand

C:\Users\Hernando\AppData\Local\Temp\ipykernel_2160\2314624634.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_df['Title'] = cleaned_df['Title'].apply(reformat_title)


In [14]:
# import warnings

# # Suppress warnings
# warnings.filterwarnings('ignore')

df = cleaned_df.reset_index(drop=True).copy()

def convert_text_to_numeric(column):
    # Remove $ and commas, replace "**" with NaN
    column = column.replace({'\$': '', ',': '', r'\*\*': np.nan}, regex=True)
    
    # Convert percentages to fractions
    column = column.replace('%', '', regex=True).astype(float) 

    # Convert to numeric
    return pd.to_numeric(column, errors='raise')

# Columns that you want to convert
columns_to_convert = ['Consumers Count', 'Total Expenditures', 'Total Authorized Services',
                      'Per Capita Expenditures', 'Per Capita Authorized Services', 'Utilized']

# Apply conversion to each column
for col in columns_to_convert:
    df[col] = convert_text_to_numeric(df[col])
    
    

def insert_totals_check_rows(df, columns_to_sum):
    # Initialize the start index for the range to sum
    start_idx = 0

    # Create a list to store new rows (to avoid changing DataFrame size during iteration)
    new_rows = []

    # Iterate through the DataFrame
    for idx, row in df.iterrows():
        if row['Category'] == 'Totals':
            
            title = row['Title']
            age = row['Age Category']
            sheet = row['Sheet Name']
            center = row['Regional Center']
            
            # Calculate sum for the specified range and columns
            sums = df.loc[start_idx:idx-1, columns_to_sum].sum()

            # Create a new row for 'Totals Check'
            totals_check_row = {'Category': 'Totals Check','Title':title,'Age Category':age, "Sheet Name":sheet, "Regional Center": center}
            totals_check_row.update(sums.to_dict())

            # Store the 'Totals Check' row and its intended insertion index
            new_rows.append((idx, totals_check_row))

            # Update the start index for the next range
            start_idx = idx + 1

    # Insert the new 'Totals Check' rows into the DataFrame
    for insert_idx, row in reversed(new_rows):
        # Insert the row
        df = pd.concat([df.iloc[:insert_idx], pd.DataFrame([row]), df.iloc[insert_idx:]]).reset_index(drop=True)

    # Reset the index of the DataFrame
    df.reset_index(drop=True, inplace=True)

    return df

# Apply the function to the dataframe
df_with_checks = insert_totals_check_rows(df, columns_to_sum)

df_with_checks['Category'] = df_with_checks['Category'].astype(str)
df_with_checks['Regional Center'] = df_with_checks['Regional Center'].astype(str)
df_with_checks['Title'] = df_with_checks['Title'].astype(str)
df_with_checks['Age Category'] = df_with_checks['Age Category'].astype(str)

# Display the first 20 rows
df_with_checks

,Category,Consumers Count,Total Expenditures,Total Authorized Services,Per Capita Expenditures,Per Capita Authorized Services,Utilized,Regional Center,Sheet Name,Title,Age Category,Total Eligible Consumers,Consumers Receiving Purchased Services,Consumers with No Purchased Services,Percent With No Purchased Services
0,American Indian or Alaska\nNative,163.0,3361984.0,4489724.0,20625.670123,27544.320552,0.748818,Alta California,POSxEth,Ethnicity or Race: Total Annual Expenditures a...,All Ages,NaN,NaN,NaN,NaN
1,Asian,3369.0,35808544.0,55766125.0,10628.834817,16552.723268,0.642120,Alta California,POSxEth,Ethnicity or Race: Total Annual Expenditures a...,All Ages,NaN,NaN,NaN,NaN
2,Black/African American,3678.0,67015072.0,94283572.0,18220.519859,25634.467675,0.710782,Alta California,POSxEth,Ethnicity or Race: Total Annual Expenditures a...,All Ages,NaN,NaN,NaN,NaN
3,Hispanic,5832.0,56732894.0,88281428.0,9727.862483,15137.419100,0.642637,Alta California,POSxEth,Ethnicity or Race: Total Annual Expenditures a...,All Ages,NaN,NaN,NaN,NaN
4,Native Hawaiian or Other\nPacific Islander,151.0,1181869.0,1904961.0,7826.949007,12615.637616,0.620416,Alta California,POSxEth,Ethnicity or Race: Total Annual Expenditures a...,All Ages,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5460,Cantonese/Mandarin\nChinese,0.0,0.0,0.0,0.000000,0.000000,0.000000,Valley Mountain,LANGxOther,Language for Residence Type: Other: Total Annu...,For age 22 years and older,NaN,NaN,NaN,NaN
5461,English,56.0,1012980.0,1661305.0,18088.922143,29666.164464,0.609749,Valley Mountain,LANGxOther,Language for Residence Type: Other: Total Annu...,For age 22 years and older,NaN,NaN,NaN,NaN
5462,Spanish,NaN,NaN,NaN,836.415000,1813.200000,0.461292,Valley Mountain,LANGxOther,Language for Residence Type: Other: Total Annu...,For age 22 years and older,NaN,NaN,NaN,NaN
5463,Totals Check,56.0,1012980.0,1661305.0,24409.241143,38319.984464,1.872709,Valley Mountain,LANGxOther,Language for Residence Type: Other: Total Annu...,For age 22 years and older,NaN,NaN,NaN,NaN


In [16]:
df_with_checks

,Category,Consumers Count,Total Expenditures,Total Authorized Services,Per Capita Expenditures,Per Capita Authorized Services,Utilized,Regional Center,Sheet Name,Title,Age Category,Total Eligible Consumers,Consumers Receiving Purchased Services,Consumers with No Purchased Services,Percent With No Purchased Services
0,American Indian or Alaska\nNative,163.0,3361984.0,4489724.0,20625.670123,27544.320552,0.748818,Alta California,POSxEth,Ethnicity or Race: Total Annual Expenditures a...,All Ages,NaN,NaN,NaN,NaN
1,Asian,3369.0,35808544.0,55766125.0,10628.834817,16552.723268,0.642120,Alta California,POSxEth,Ethnicity or Race: Total Annual Expenditures a...,All Ages,NaN,NaN,NaN,NaN
2,Black/African American,3678.0,67015072.0,94283572.0,18220.519859,25634.467675,0.710782,Alta California,POSxEth,Ethnicity or Race: Total Annual Expenditures a...,All Ages,NaN,NaN,NaN,NaN
3,Hispanic,5832.0,56732894.0,88281428.0,9727.862483,15137.419100,0.642637,Alta California,POSxEth,Ethnicity or Race: Total Annual Expenditures a...,All Ages,NaN,NaN,NaN,NaN
4,Native Hawaiian or Other\nPacific Islander,151.0,1181869.0,1904961.0,7826.949007,12615.637616,0.620416,Alta California,POSxEth,Ethnicity or Race: Total Annual Expenditures a...,All Ages,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5460,Cantonese/Mandarin\nChinese,0.0,0.0,0.0,0.000000,0.000000,0.000000,Valley Mountain,LANGxOther,Language for Residence Type: Other: Total Annu...,For age 22 years and older,NaN,NaN,NaN,NaN
5461,English,56.0,1012980.0,1661305.0,18088.922143,29666.164464,0.609749,Valley Mountain,LANGxOther,Language for Residence Type: Other: Total Annu...,For age 22 years and older,NaN,NaN,NaN,NaN
5462,Spanish,NaN,NaN,NaN,836.415000,1813.200000,0.461292,Valley Mountain,LANGxOther,Language for Residence Type: Other: Total Annu...,For age 22 years and older,NaN,NaN,NaN,NaN
5463,Totals Check,56.0,1012980.0,1661305.0,24409.241143,38319.984464,1.872709,Valley Mountain,LANGxOther,Language for Residence Type: Other: Total Annu...,For age 22 years and older,NaN,NaN,NaN,NaN


In [17]:
df_with_checks.to_csv('cleaned_data2.csv', index=False)